In [17]:
import tensorflow as tf
import sounddevice as sd
import scipy.signal
import librosa
import numpy as np

# 모델 불러오기
model = tf.keras.models.load_model('voice_recognition_model.h5')

y, sr = librosa.load("./generated_noise_0.wav")

hop_length = int(sr * 0.010)  # 프레임 크기 (여기서는 10ms)
n_fft = int(sr * 0.025)  # FFT 크기 (여기서는 25ms)
spectrogram = np.abs(librosa.stft(y, hop_length=hop_length, n_fft=n_fft))
print(spectrogram.shape)
prediction = model.predict(spectrogram.reshape(1, 276, 301, 1))
prediction = [[format(num, '.2f') for num in sublist] for sublist in prediction]
print(prediction)

(276, 301)
1/1 [==============================] - 0s 45ms/step
[['1.00', '0.00']]


2024-01-03 16:57:25.577680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [18]:
import os
import librosa
import numpy as np
from scipy.ndimage import zoom
import tqdm
from sklearn.decomposition import PCA

# .wav 파일이 저장된 폴더
folder_path = './testset/'

# .wav 파일을 스펙트로그램으로 변환하여 저장할 리스트
spectrograms = []
labels = []

# '0'과 '1' 폴더를 순회
for label in ['0', '1']:
    label_folder_path = os.path.join(folder_path, label)
    print(label)

    # folder_path 내의 모든 파일을 순회
    for file_name in os.listdir(label_folder_path):
        # .wav 파일만 처리
        if file_name.endswith('.wav'):
            # 오디오 파일 읽기
            y, sr = librosa.load(os.path.join(label_folder_path, file_name))

            hop_length = int(sr * 0.010)  # 프레임 크기 (여기서는 10ms)
            n_fft = int(sr * 0.025)  # FFT 크기 (여기서는 25ms)
            spectrogram = np.abs(librosa.stft(y, hop_length=hop_length, n_fft=n_fft))

            # # 오디오 파일을 스펙트로그램으로 변환
            # D = librosa.stft(y)
            # spectrogram = librosa.amplitude_to_db(np.abs(D), ref=np.max)

            # 리스트에 스펙트로그램 추가
            spectrograms.append(spectrogram)

            # 레이블 추가
            labels.append(int(label))

# numpy 배열로 변환
X_test = np.array(spectrograms)
Y_test = np.array(labels)

X_test = np.expand_dims(X_test, -1)



print(X_test.shape)
print(Y_test.shape)



0
1
(200, 276, 301, 1)
(200,)


In [20]:
results = model.evaluate(X_test, Y_test, batch_size=32)
print(results)

7/7 [==============================] - 0s 50ms/step - loss: 1.4281e-04 - accuracy: 1.0000
[0.00014281136100180447, 1.0]


In [108]:
import tensorflow as tf
import sounddevice as sd
import numpy as np

# 모델 불러오기
model = tf.keras.models.load_model('voice_recognition_model.h5')

arrayList = []
# 오디오 스트림 콜백 함수
def audio_callback(indata, frames, time, status):
    indata = indata[0:201]
    # arrayList에 indata 추가
    arrayList.append(indata)
    if len(arrayList) == 301:
        arr = np.array(arrayList).transpose()
    # 예측
        prediction = model.predict(arr)
        prediction = [[format(num, '.2f') for num in sublist] for sublist in prediction]
    # 결과 출력
        answer = ["인식 안됨", "인식 됨"]
        print('결과:', answer[np.argmax(prediction)])
        arrayList.clear()
    # print('인식 결과:', prediction)

# 오디오 스트림 시작
stream = sd.InputStream(callback=audio_callback, channels=1, samplerate=43000)
stream.start()

# 입력 대기
input('목소리 인식 중... (끝내려면 Enter 키를 누르세요)')

# 오디오 스트림 종료
stream.stop()
stream.close()

1/1 [==============================] - 0s 68ms/step


2024-01-03 18:26:23.800725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


결과: 인식 안됨
1/1 [==============================] - 0s 16ms/step
결과: 인식 안됨
1/1 [==============================] - 0s 11ms/step
결과: 인식 됨
1/1 [==============================] - 0s 16ms/step
결과: 인식 안됨
1/1 [==============================] - 0s 13ms/step
결과: 인식 됨
1/1 [==============================] - 0s 13ms/step
결과: 인식 안됨
1/1 [==============================] - 0s 11ms/step
결과: 인식 안됨
1/1 [==============================] - 0s 13ms/step
결과: 인식 됨
1/1 [==============================] - 0s 11ms/step
결과: 인식 됨
1/1 [==============================] - 0s 12ms/step
결과: 인식 안됨
1/1 [==============================] - 0s 12ms/step
결과: 인식 됨
1/1 [==============================] - 0s 13ms/step
결과: 인식 안됨
1/1 [==============================] - 0s 12ms/step
결과: 인식 됨
1/1 [==============================] - 0s 10ms/step
결과: 인식 안됨
1/1 [==============================] - 0s 13ms/step
결과: 인식 안됨
1/1 [==============================] - 0s 11ms/step
결과: 인식 됨
1/1 [==============================] - 0s 11ms/step
결과: 인식 안됨
1/1 [

In [76]:
y, sr = librosa.load(os.path.join("./test_output.wav"), sr=16000)
print(y.shape)

hop_length = int(sr * 0.010)  # 프레임 크기 (여기서는 10ms)
n_fft = int(sr * 0.025)  # FFT 크기 (여기서는 25ms)
spectrogram = np.abs(librosa.stft(y, hop_length=hop_length, n_fft=n_fft))

model = tf.keras.models.load_model('voice_recognition_model.h5')

prediction = model.predict(spectrogram.reshape(1, 276, 301, 1))
prediction = [[format(num, '.2f') for num in sublist] for sublist in prediction]
print(prediction)

(48000,)


ValueError: cannot reshape array of size 60501 into shape (1,276,301,1)